In [120]:
import numpy as np
import os

In [270]:
# Get labels
y = np.loadtxt("data/labels.csv", delimiter=',', dtype='int')    # 0 = typical, 1 = ASD

# Get data
num_files = len(os.listdir('./data/networks'))
data_lst = []
for i in range(num_files):
    x = np.tril(np.loadtxt(f"data/networks/train_{i}.csv", delimiter=','), -1)[1:,:-1].flatten()
    threshold = np.percentile(x, 99)
    x[x < threshold] = 0
    data_lst.append(x)

X = np.array(data_lst)
print(X.shape)

(120, 1156)


In [271]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [278]:
svc = SVC()
parameters = {'C':[.1, 1., 10.]}
gs = GridSearchCV(svc, parameters, cv=5)

In [279]:
gs.fit(X, y)
mean_scores = gs.cv_results_['mean_test_score']
scores_std = gs.cv_results_['std_test_score']

print(mean_scores)
print(scores_std)

[0.51666667 0.575      0.49166667]
[0.02041241 0.06666667 0.04082483]


In [273]:
scores = []
for i in range(10):
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scores.append(accuracy_score(y_test, y_pred))
print(np.mean(scores))


0.5325


In [267]:
from sklearn.pipeline import Pipeline
from nilearn.connectome import ConnectivityMeasure
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

kinds = ['correlation', 'partial correlation', 'tangent']

pipe = Pipeline(
    [('connectivity', ConnectivityMeasure(vectorize=True)),
     ('classifier', GridSearchCV(LinearSVC(), {'C': [.1, 1., 10.]}, cv=5))])

param_grid = [
    {'classifier': [DummyClassifier('most_frequent')]},
    {'connectivity__kind': kinds}
]

/Users/vsharm/Desktop/Practice/mri-test/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass strategy=most_frequent as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [268]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder

In [269]:
cv = StratifiedShuffleSplit(n_splits=30, random_state=0, test_size=10)
gs = GridSearchCV(pipe, param_grid, scoring='accuracy', cv=cv, verbose=1,
                  refit=False, n_jobs=8)

In [265]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder

groups = y
classes = LabelEncoder().fit_transform(groups)

cv = StratifiedShuffleSplit(n_splits=30, random_state=0, test_size=10)
gs = GridSearchCV(pipe, param_grid, scoring='accuracy', cv=cv, verbose=1,
                  refit=False, n_jobs=8)
gs.fit(X, y)
mean_scores = gs.cv_results_['mean_test_score']
scores_std = gs.cv_results_['std_test_score']
print(mean_scores)
print(scores_std)

Fitting 30 folds for each of 4 candidates, totalling 120 fits
[nan nan nan nan]
[nan nan nan nan]


/Users/vsharm/Desktop/Practice/mri-test/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(
